In [2]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt 
from datetime import timedelta
%matplotlib inline

#from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score

import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

In [3]:
labels = pd.read_csv("../../data/labels_training_set.csv")
df = pd.read_csv("../../data/events_up_to_01062018.csv")

C:\Users\seba\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_persons = pd.read_csv("../../data/trocafone_kaggle_test.csv")

In [5]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["month"] = df["timestamp"].dt.month
df["day"] = df["timestamp"].dt.day
df['dayofyear'] = df['timestamp'].dt.dayofyear

In [6]:
labels = pd.read_csv('../../data/labels_training_set.csv') #las personas de las cuales tengo Info
personas =pd.read_csv('../../data/trocafone_kaggle_test.csv') #las personas a las que le tengo que predecir

In [7]:
df_features = pd.read_csv('../features_csv/features_finales_seba.csv')

In [19]:
#Datos para entrenar

labels_f = pd.merge( df_features, labels, on = 'person', how = 'inner')

In [20]:
todas_las_personas = df[["person"]].drop_duplicates(subset="person")
df_con_labels = pd.merge(df, labels, on='person', how='inner')

In [21]:
y = labels_f[['label']]
X = labels_f.drop(columns=['person', 'label'])

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 123)

In [22]:
def prueba_svc(X, y, pesos={0: 1, 1: 20}):
    model = SVC(C=0.5, kernel='linear', probability=True, class_weight=pesos)
      
    X_train, X_test, y_train, y_test = train_test_split(X, y['label'], test_size= 0.2, random_state= 123)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    preds_prob = model.predict_proba(X_test)[:,1]
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, preds)
    area_debajo_de_curva = roc_auc_score(y_test, preds_prob)
    matriz_de_confusion = confusion_matrix(y_test, preds)
    print('train acurracy: ', train_accuracy)
    print('test acurracy: ', test_accuracy)
    print('Matriz de confusión: ')
    print(matriz_de_confusion)
    print('Área bajo la curva: ', area_debajo_de_curva)

In [23]:
def prueba_random_forest(X, y, pesos={0: 1, 1: 15}):
    clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0, class_weight = pesos)
    X_train, X_test, y_train, y_test = train_test_split(X, y['label'], test_size= 0.2, random_state= 123)
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    preds_prob = clf.predict_proba(X_test)[:,1]
    train_accuracy = accuracy_score(y_train, clf.predict(X_train))
    test_accuracy = accuracy_score(y_test, preds)
    area_debajo_de_curva = roc_auc_score(y_test, preds_prob)
    matriz_de_confusion = confusion_matrix(y_test, preds)
    print('train acurracy: ', train_accuracy)
    print('test acurracy: ', test_accuracy)
    print('Matriz de confusión: ')
    print(matriz_de_confusion)
    print('Área bajo la curva: ', area_debajo_de_curva)

In [24]:
def prueba_decision_tree(X, y, pesos={0: 1, 1: 15}):
    model = DecisionTreeClassifier(max_depth=5, random_state=123, class_weight = {0: 1, 1: 10}, min_samples_split=10)
    X_train, X_test, y_train, y_test = train_test_split(X, y['label'], test_size= 0.2, random_state= 123)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    preds_prob = model.predict_proba(X_test)[:,1]
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, preds)
    area_debajo_de_curva = roc_auc_score(y_test, preds_prob)
    matriz_de_confusion = confusion_matrix(y_test, preds)
    print('train acurracy: ', train_accuracy)
    print('test acurracy: ', test_accuracy)
    print('Matriz de confusión: ')
    print(matriz_de_confusion)
    print('Área bajo la curva: ', area_debajo_de_curva)

In [25]:
def prueba_bagging_classifier(X, y, pesos={0: 1, 1: 15}):
    model = BaggingClassifier(DecisionTreeClassifier(max_depth=5, random_state=0, class_weight = {0: 1, 1: 10}))
    X_train, X_test, y_train, y_test = train_test_split(X, y['label'], test_size= 0.2, random_state= 123)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    preds_prob = model.predict_proba(X_test)[:,1]
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, preds)
    area_debajo_de_curva = roc_auc_score(y_test, preds_prob)
    matriz_de_confusion = confusion_matrix(y_test, preds)
    print('train acurracy: ', train_accuracy)
    print('test acurracy: ', test_accuracy)
    print('Matriz de confusión: ')
    print(matriz_de_confusion)
    print('Área bajo la curva: ', area_debajo_de_curva)

In [26]:
def prueba_ada_boost(X, y):
    model = AdaBoostClassifier( learning_rate = 0.1, n_estimators = 100)
    X_train, X_test, y_train, y_test = train_test_split(X, y['label'], test_size= 0.2, random_state= 123)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    preds_prob = model.predict_proba(X_test)[:,1]
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, preds)
    area_debajo_de_curva = roc_auc_score(y_test, preds_prob)
    matriz_de_confusion = confusion_matrix(y_test, preds)
    print('train acurracy: ', train_accuracy)
    print('test acurracy: ', test_accuracy)
    print('Matriz de confusión: ')
    print(matriz_de_confusion)
    print('Área bajo la curva: ', area_debajo_de_curva)

In [27]:
def prueba_xgboost(X, y):
    model = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y['label'], test_size= 0.2, random_state= 123)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    preds_prob = model.predict_proba(X_test)[:,1]
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, preds)
    area_debajo_de_curva = roc_auc_score(y_test, preds_prob)
    matriz_de_confusion = confusion_matrix(y_test, preds)
    print('train acurracy: ', train_accuracy)
    print('test acurracy: ', test_accuracy)
    print('Matriz de confusión: ')
    print(matriz_de_confusion)
    print('Área bajo la curva: ', area_debajo_de_curva)

In [28]:
def prueba_xgboost_oversampling(df, cant_unos):
   
    df_train = pd.merge(df, labels, on="person", how="inner")\
            .drop(columns = ['person'])
    
    # Separamos la variable a predecir

    X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]


    #Creamos set de entrenamiento y test

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
    
    cant_ceros = np.sum(y_train == 0)
    
    ros = RandomOverSampler(random_state=2, sampling_strategy = {0: cant_ceros, 1: cant_unos})
    X_train_res, y_train_res = ros.fit_sample(X_train, y_train)
    X_train_res_df = pd.DataFrame(X_train_res)
    X_train_res_df.columns = X_test.columns

    #Instanciamos el regresor de XGBoost

    xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)

    #Entrenamos

    xg_reg.fit(X_train_res_df, y_train_res)

    # Predecimos

    preds = xg_reg.predict(X_test)
    proba = xg_reg.predict_proba(X_test)[:,1]
    
    # Vemos que onda

    train_accuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
    test_accuracy = accuracy_score(y_test, preds)
    matriz_de_confusion = confusion_matrix(y_test, preds)
    area_debajo_de_curva = roc_auc_score(y_test, proba)

    print("Train accuracy: ", train_accuracy)
    print("Test acuracy: ", test_accuracy)
    print("ROC auc score: ", area_debajo_de_curva)
    print("Confusion matrix: ")
    print(matriz_de_confusion)
 

In [29]:
prueba_xgboost(X, y)

train acurracy:  0.8790161612259352
test acurracy:  0.872263713623487
Matriz de confusión: 
[[3264  421]
 [  75  123]]
Área bajo la curva:  0.8727457752559515


In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
np.sum(y_train == 1), np.sum(y_train == 0)

(label    782
 dtype: int64, label    14749
 dtype: int64)

In [31]:
prueba_xgboost_oversampling(df_features, 900)

Train accuracy:  0.8603744648220334
Test acuracy:  0.8599021375225341
ROC auc score:  0.8728718665625043
Confusion matrix: 
[[3210  475]
 [  69  129]]


In [32]:
prueba_ada_boost(X, y)

train acurracy:  0.9495847015646127
test acurracy:  0.950296162760752
Matriz de confusión: 
[[3685    0]
 [ 193    5]]
Área bajo la curva:  0.8706008524868769


In [33]:
prueba_bagging_classifier(X,y)

train acurracy:  0.8430236301590368
test acurracy:  0.8411022405356683
Matriz de confusión: 
[[3133  552]
 [  65  133]]
Área bajo la curva:  0.8666673519455065


In [34]:
prueba_decision_tree(X, y)

train acurracy:  0.8181057240357994
test acurracy:  0.8174092196755086
Matriz de confusión: 
[[3035  650]
 [  59  139]]
Área bajo la curva:  0.813254663322506


In [35]:
prueba_random_forest(X, y)

train acurracy:  0.8207456055630674
test acurracy:  0.8241050733968581
Matriz de confusión: 
[[3049  636]
 [  47  151]]
Área bajo la curva:  0.8682174526814962


# Ensamble con promedio

In [37]:
#Prueba con oversampling

pesos={0: 1, 1: 15}
model1 = RandomForestClassifier(n_estimators=16, max_depth=5, random_state=0, class_weight = pesos)

model2 = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)


model3 = DecisionTreeClassifier(max_depth=5, random_state=123, class_weight = {0: 1, 1: 10})
model4=  BaggingClassifier(DecisionTreeClassifier(max_depth=5, random_state=0, class_weight = {0: 1, 1: 10}))
model5 = AdaBoostClassifier( learning_rate = 0.1, n_estimators = 100)

df_train = pd.merge(df_features, labels, on="person", how="inner")\
            .drop(columns = ['person'])
    
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 123)

cant_unos = 900
ros = RandomOverSampler(random_state=2, sampling_strategy = {0: np.sum(y_train == 0), 1: cant_unos})
X_train_res, y_train_res = ros.fit_sample(X_train, y_train)
X_train_res_df = pd.DataFrame(X_train_res)
X_train_res_df.columns = X_test.columns

model1.fit(X_train, y_train)
model2.fit(X_train_res_df, y_train_res)
model3.fit(X_train, y_train)
model4.fit(X_train_res_df, y_train_res)
model5.fit(X_train_res_df, y_train_res)

pred1=model1.predict_proba(X_test)[:,1]
pred2=model2.predict_proba(X_test)[:,1]
pred3=model3.predict_proba(X_test)[:,1]
pred4 = model4.predict_proba(X_test)[:,1]
pred5 = model5.predict_proba(X_test)[:,1]


preds_prob = (pred1 + pred2 + pred5)/3

area_debajo_de_curva = roc_auc_score(y_test, preds_prob)
print('Área bajo la curva: ', area_debajo_de_curva)

Área bajo la curva:  0.8756534133739019


In [ ]:
#Esto seria para kaggle

pesos={0: 1, 1: 15}
model1 = RandomForestClassifier(n_estimators=16, max_depth=5, random_state=0, class_weight = pesos)

model2 = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)

df_train = pd.merge(df_features, labels, on="person", how="inner")\
            .drop(columns = ['person'])

datos = pd.merge(df_features, personas, on='person', how='inner').filter(items= list(df_train.columns) + ['person'])
df_final_sin_persons = datos.drop(columns='person')
    
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

cant_unos = 1098
ros = RandomOverSampler(random_state=2, sampling_strategy = {0: np.sum(y == 0), 1: cant_unos})
X_res, y_res = ros.fit_sample(X, y)
X_res_df = pd.DataFrame(X_res)
X_res_df.columns = X.columns

model3 = DecisionTreeClassifier(max_depth=5, random_state=123, class_weight = {0: 1, 1: 10})
model4=  BaggingClassifier(DecisionTreeClassifier(max_depth=5, random_state=0, class_weight = {0: 1, 1: 10}))
model5 = AdaBoostClassifier( learning_rate = 0.1, n_estimators = 100)

model1.fit(X, y)
model2.fit(X_res_df, y_res)
model3.fit(X, y)
model4.fit(X_res_df, y_res)
model5.fit(X_res_df, y_res)

pred1 = pd.Series(model1.predict_proba(df_final_sin_persons)[:,1])
pred2 = pd.Series(model2.predict_proba(df_final_sin_persons)[:,1])
pred5 = pd.Series(model5.predict_proba(df_final_sin_persons)[:,1])

preds_prob = (pred1 + pred2 + pred5)/3

predicciones = datos[['person']]
predicciones['label'] = preds_prob
predicciones.to_csv('predicciones.csv', encoding='utf-8', index=False)

print(predicciones.shape[0] == 19415)

In [ ]:
predicciones.head()

In [ ]:
def kaggle_promedio(df):
    
    # Separamos la variable a predecir

    df_train = pd.merge(df, labels, on="person", how="inner")\
            .drop(columns = ['person'])
    
    X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]
    
    datos = pd.merge(df, personas, on='person', how='inner').filter(items= list(df_train.columns) + ['person'])
    df_final_sin_persons = datos.drop(columns='person')

    # Entreno
    pesos={0: 1, 1: 15}

    model1 = RandomForestClassifier(n_estimators=16, max_depth=5, random_state=0, class_weight = pesos)

    model2 = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)

    model1.fit(X,y)
    model2.fit(X,y)


    pred1 = pd.Series(model1.predict_proba(df_final_sin_persons)[:,1])
    pred2 = pd.Series(model2.predict_proba(df_final_sin_persons)[:,1])

    proba=(0.8*pred2 + 0.2*pred1)
    predicciones = datos[['person']]
    predicciones['label'] = proba
    predicciones.to_csv('predicciones.csv', encoding='utf-8', index=False)

    print(predicciones.shape[0] == 19415)

In [ ]:
def kaggle_oversample(df):
    
    # Separamos la variable a predecir

    df_train = pd.merge(df, labels, on="person", how="inner")\
            .drop(columns = ['person'])
    
    X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]
    
    cant_unos = np.sum(y == 1) + 91
    cant_ceros = np.sum(y == 0)
    ros = RandomOverSampler(random_state=2, sampling_strategy = {0: cant_ceros, 1: cant_unos})
    X_ro, y_ro = ros.fit_sample(X, y)
    X_ro = pd.DataFrame(X_ro)
    X_ro.columns = X.columns
    
    datos = pd.merge(df, personas, on='person', how='inner').filter(items= list(df_train.columns) + ['person'])
    df_final_sin_persons = datos.drop(columns='person')

    # Entreno
   
    xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)

    xg_reg.fit(X_ro, y_ro)

    proba = pd.Series(xg_reg.predict_proba(df_final_sin_persons)[:,1])

    predicciones = datos[['person']]
    predicciones['label'] = proba
    predicciones.to_csv('predicciones.csv', encoding='utf-8', index=False)

    print(predicciones.shape[0] == 19415)

In [ ]:
def kaggle(df):
    
    # Separamos la variable a predecir

    df_train = pd.merge(df, labels, on="person", how="inner")\
            .drop(columns = ['person'])
    
    X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]
    
    datos = pd.merge(df, personas, on='person', how='inner').filter(items= list(df_train.columns) + ['person'])
    df_final_sin_persons = datos.drop(columns='person')

    # Entreno
   
    xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)

    xg_reg.fit(X, y)

    proba = pd.Series(xg_reg.predict_proba(df_final_sin_persons)[:,1])

    predicciones = datos[['person']]
    predicciones['label'] = proba
    predicciones.to_csv('predicciones.csv', encoding='utf-8', index=False)

    print(predicciones.shape[0] == 19415)

In [ ]:
kaggle_promedio(df_features)